# Sudoku-15-Infer-Python : Resolution Probabiliste avec NumPyro

**Navigation** : [<< Choco](Sudoku-11-Choco-Python.ipynb) | [Index](README.md) | [Neural Network >>](Sudoku-16-NeuralNetwork-Python.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Comprendre** les limites de la programmation probabiliste pour les CSP discrets
2. **Implementer** un modele probabiliste avec distributions Dirichlet pour les cellules
3. **Utiliser** l'inference variationnelle et l'enumeration pour l'inférence
4. **Comparer** les approches iteratives vs inference directe

**Duree estimee** : ~40 min | **Prerequis** : Sudoku-0 Environment, notions de probabilites

---

## Introduction : Programmation Probabiliste et Sudoku

Ce notebook est l'equivalent Python du notebook C# Infer.NET. Cependant, il y a une **difference fondamentale** :

| Aspect | Infer.NET (C#) | NumPyro/PyMC (Python) |
|--------|----------------|----------------------|
| **Contraintes** | Dures (via `ConstrainFalse`) | Douces (via penalites) |
| **Algorithme** | Expectation Propagation | VI / Enumeration / MCMC |
| **Variables discretes** | Natif | Enumeration requise |
| **Performance Sudoku** | ~1-5s | ~1-10s |

### Pourquoi etudier cette approche si elle n'est pas optimale ?

1. **Pedagogie** : Comprendre comment modeliser des problemes discrets probabilistement
2. **Comparaison** : Voir les differences entre contraintes dures et douces
3. **Hybridation** : L'approche iterative peut etre combinee avec de la propagation de contraintes

## 1. Installation et Imports

In [1]:
# Installation des dependances (si necessaire)
import subprocess
import sys

def install_if_needed(package, import_name=None):
    import_name = import_name or package
    try:
        __import__(import_name)
        print(f"{package} deja installe")
    except ImportError:
        print(f"Installation de {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

install_if_needed("numpyro")
install_if_needed("jax", "jax")
install_if_needed("optax")  # Pour les optimiseurs avances

import jax
import jax.numpy as jnp
import numpy as np
import numpyro
import numpyro.distributions as dist
from numpyro.infer import SVI, Trace_ELBO, Predictive
from numpyro.optim import Adam
from jax import random, jit, vmap
import time
from typing import List, Tuple, Optional

# Configuration JAX
print(f"JAX version: {jax.__version__}")
print(f"NumPyro version: {numpyro.__version__}")
print(f"Backend: {jax.devices()}")

numpyro deja installe
jax deja installe
optax deja installe
JAX version: 0.9.0.1
NumPyro version: 0.20.0
Backend: [CpuDevice(id=0)]


## 2. Utilitaires de Chargement et Validation

In [2]:
def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    """Charge les puzzles depuis un fichier."""
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

def puzzle_to_grid(puzzle_str: str) -> List[List[int]]:
    """Convertit une chaine de 81 caracteres en grille 9x9."""
    return [[int(puzzle_str[i * 9 + j]) if puzzle_str[i * 9 + j] in '123456789' else 0 
             for j in range(9)] for i in range(9)]

def grid_to_flat(grid: List[List[int]]) -> jnp.ndarray:
    """Convertit une grille 2D en tableau 1D JAX."""
    return jnp.array([cell for row in grid for cell in row])

def flat_to_grid(flat: jnp.ndarray) -> List[List[int]]:
    """Convertit un tableau 1D en grille 2D."""
    flat = np.array(flat)
    return [[int(flat[i * 9 + j]) for j in range(9)] for i in range(9)]

def verify_solution(grid: List[List[int]]) -> bool:
    """Verifie qu'une solution Sudoku est valide."""
    # Verifier les lignes
    for row in grid:
        if sorted(row) != list(range(1, 10)):
            return False
    # Verifier les colonnes
    for c in range(9):
        col = [grid[r][c] for r in range(9)]
        if sorted(col) != list(range(1, 10)):
            return False
    # Verifier les blocs 3x3
    for br in range(3):
        for bc in range(3):
            box = []
            for r in range(3):
                for c in range(3):
                    box.append(grid[br*3+r][bc*3+c])
            if sorted(box) != list(range(1, 10)):
                return False
    return True

def count_errors(grid: List[List[int]]) -> int:
    """Compte le nombre d'erreurs dans une grille."""
    errors = 0
    # Erreurs dans les lignes
    for row in grid:
        seen = set()
        for cell in row:
            if cell in seen:
                errors += 1
            elif cell > 0:
                seen.add(cell)
    # Erreurs dans les colonnes
    for c in range(9):
        seen = set()
        for r in range(9):
            cell = grid[r][c]
            if cell in seen:
                errors += 1
            elif cell > 0:
                seen.add(cell)
    # Erreurs dans les blocs
    for br in range(3):
        for bc in range(3):
            seen = set()
            for r in range(3):
                for c in range(3):
                    cell = grid[br*3+r][bc*3+c]
                    if cell in seen:
                        errors += 1
                    elif cell > 0:
                        seen.add(cell)
    return errors

def print_grid(grid: List[List[int]], title: str = ""):
    """Affiche une grille Sudoku."""
    if title:
        print(f"\n{title}")
    print("-" * 25)
    for i, row in enumerate(grid):
        if i % 3 == 0 and i > 0:
            print("|" + "-" * 23 + "|")
        line = "| "
        for j, cell in enumerate(row):
            if j % 3 == 0 and j > 0:
                line += "| "
            line += f"{cell if cell > 0 else ' '} "
        line += "|"
        print(line)
    print("-" * 25)

# Charger quelques puzzles pour les tests
import os

# Trouver le bon chemin pour les puzzles
possible_paths = [
    "Puzzles/Sudoku_Easy51.txt",  # Depuis le notebook
    "MyIA.AI.Notebooks/Sudoku/Puzzles/Sudoku_Easy51.txt",  # Depuis la racine
]

puzzles = []
for path in possible_paths:
    try:
        puzzles = load_puzzles(path, max_puzzles=5)
        if puzzles:
            print(f"{len(puzzles)} puzzles charges depuis {path}")
            break
    except FileNotFoundError:
        continue

if not puzzles:
    # Fallback: creer un puzzle de test manuellement
    print("Utilisation d'un puzzle de test par defaut")
    test_puzzle_str = "900200543100063025508407060026309001057010290090670530240530600705200304080041950"
    puzzles = [test_puzzle_str]

5 puzzles charges depuis MyIA.AI.Notebooks/Sudoku/Puzzles/Sudoku_Easy51.txt


## 3. Modele Probabiliste Simple (Approche Naive)

La premiere approche utilise des distributions categorielles pour chaque cellule, avec des contraintes douces encodees comme des penalites.

In [3]:
def compute_constraint_penalty(cell_values: jnp.ndarray) -> jnp.ndarray:
    """
    Calcule les penalites de contraintes pour une grille Sudoku.
    
    Args:
        cell_values: Tableau de shape (81,) avec des valeurs 0-8 (index)
    
    Returns:
        Penalite totale (plus elle est basse, meilleure est la solution)
    """
    penalty = 0.0
    
    # Penalites pour les lignes
    for r in range(9):
        row_vals = cell_values[r * 9:(r + 1) * 9]
        # Compter les occurrences de chaque valeur
        for v in range(9):
            count = jnp.sum((row_vals == v).astype(jnp.float32))
            penalty = penalty + jnp.maximum(0, count - 1) ** 2
    
    # Penalites pour les colonnes
    for c in range(9):
        col_vals = cell_values[c::9]
        for v in range(9):
            count = jnp.sum((col_vals == v).astype(jnp.float32))
            penalty = penalty + jnp.maximum(0, count - 1) ** 2
    
    # Penalites pour les blocs 3x3
    for br in range(3):
        for bc in range(3):
            indices = []
            for i in range(3):
                for j in range(3):
                    indices.append((br * 3 + i) * 9 + (bc * 3 + j))
            box_vals = cell_values[jnp.array(indices)]
            for v in range(9):
                count = jnp.sum((box_vals == v).astype(jnp.float32))
                penalty = penalty + jnp.maximum(0, count - 1) ** 2
    
    return penalty


def simple_sudoku_model(initial_grid: jnp.ndarray, temperature: float = 1.0):
    """
    Modele probabiliste simple pour Sudoku avec contraintes douces.
    
    Args:
        initial_grid: Grille initiale (81,) avec 0 pour les cellules vides
        temperature: Parametre de temperature pour les penalites
    """
    # Distribution a priori uniforme pour chaque cellule
    # Les cellules connues seront fixees par l'observation
    
    cell_probs = numpyro.sample(
        "cell_probs",
        dist.Dirichlet(jnp.ones((81, 9))).to_event(1)
    )
    
    # Fixer les cellules connues avec des masses de Dirichlet
    epsilon = 1e-7
    fixed_prob = 1.0 - 8 * epsilon
    
    for i in range(81):
        if initial_grid[i] > 0:
            # Creer une distribution point-masse pour la valeur connue
            one_hot = jnp.zeros(9).at[initial_grid[i] - 1].set(fixed_prob)
            one_hot = one_hot.at[jnp.arange(9) != initial_grid[i] - 1].set(epsilon)
            cell_probs = cell_probs.at[i].set(one_hot)
    
    # Echantillonner les valeurs des cellules
    cell_values = numpyro.sample(
        "cell_values",
        dist.Categorical(probs=cell_probs).to_event(1)
    )
    
    # Calculer la penalite de contraintes
    penalty = compute_constraint_penalty(cell_values)
    
    # Observer une penalite faible (soft constraint)
    numpyro.sample("obs", dist.Normal(0, temperature), obs=penalty * 0.1)
    
    return cell_probs, cell_values

### Interpretation du modele simple

Ce modele presente plusieurs limitations importantes :

| Aspect | Probleme | Consequence |
|--------|----------|-------------|
| **Contraintes douces** | Les penalites ne garantissent pas l'absence de conflits | Solutions souvent invalides |
| **Espace de recherche** | 9^81 configurations possibles | Convergence lente |
| **Independance** | Les cellules sont echantillonnees independamment | Ne capture pas les correlations |

Ce modele est **educatif** mais **pas pratique** pour resoudre des Sudokus.

## 4. Solveur Iteratif avec Dirichlet (Approche Robuste)

Inspire du notebook Infer.NET, nous implementons un solveur iteratif qui :
1. Calcule les distributions de probabilite pour chaque cellule
2. Fixe les cellules avec la plus haute confiance
3. Repete jusqu'a resolution complete

In [4]:
class IterativeProbabilisticSolver:
    """
    Solveur Sudoku iteratif base sur l'inference probabiliste.
    
    Cette approche imite le IterativeSudokuModel du notebook Infer.NET C#.
    """
    
    def __init__(self, n_iterations: int = 100, cells_per_iter: int = 1, 
                 lr: float = 0.05, confidence_threshold: float = 0.9):
        self.n_iterations = n_iterations
        self.cells_per_iter = cells_per_iter
        self.lr = lr
        self.confidence_threshold = confidence_threshold
    
    def solve(self, grid: List[List[int]]) -> Tuple[List[List[int]], dict]:
        """
        Resout un Sudoku en fixant iterativement les cellules les plus certaines.
        
        Returns:
            Tuple (solution, metadata) ou metadata contient des infos sur la resolution
        """
        grid = [row[:] for row in grid]  # Copie profonde
        metadata = {
            'iterations': 0,
            'cells_fixed': [],
            'confidences': [],
            'converged': False
        }
        
        n_empty = sum(1 for row in grid for cell in row if cell == 0)
        
        while n_empty > 0:
            metadata['iterations'] += 1
            
            # Obtenir les probabilites pour chaque cellule
            probs = self._infer_probabilities(grid)
            
            # Trouver les meilleures cellules a fixer
            best_cells = self._get_best_cells(probs, grid)
            
            if not best_cells:
                # Aucune cellule avec confiance suffisante
                break
            
            # Fixer les N meilleures cellules
            fixed_this_round = 0
            for idx, value, confidence in best_cells[:self.cells_per_iter]:
                row, col = idx // 9, idx % 9
                if grid[row][col] == 0:
                    grid[row][col] = value + 1  # Convertir 0-indexe en 1-indexe
                    n_empty -= 1
                    fixed_this_round += 1
                    metadata['cells_fixed'].append((row, col, value + 1, confidence))
                    metadata['confidences'].append(confidence)
            
            if fixed_this_round == 0:
                break
            
            # Limite de securite
            if metadata['iterations'] > 200:
                break
        
        metadata['converged'] = (n_empty == 0)
        return grid, metadata
    
    def _infer_probabilities(self, grid: List[List[int]]) -> jnp.ndarray:
        """
        Utilise l'inference variationnelle pour obtenir les probabilites des cellules.
        """
        # Guide (distribution variationnelle)
        def guide(initial_grid):
            # Parametres de concentration Dirichlet apprenables
            alpha = numpyro.param(
                "alpha",
                jnp.ones((81, 9)),
                constraint=dist.constraints.positive
            )
            numpyro.sample("cell_probs", dist.Dirichlet(alpha).to_event(1))
        
        # Preparation de la grille
        initial_flat = grid_to_flat(grid)
        
        # Fixer les cellules connues dans le guide
        epsilon = 1e-7
        fixed_prob = 1.0 - 8 * epsilon
        alpha_init = jnp.ones((81, 9))
        
        for i in range(81):
            if initial_flat[i] > 0:
                # Masses de Dirichlet pour les cellules connues
                alpha_init = alpha_init.at[i, :].set(epsilon)
                alpha_init = alpha_init.at[i, initial_flat[i] - 1].set(fixed_prob * 1000)
        
        # SVI
        rng_key = random.PRNGKey(0)
        optimizer = Adam(step_size=self.lr)
        
        def model_wrapper(g):
            return simple_sudoku_model(g, temperature=0.5)
        
        svi = SVI(
            model_wrapper,
            guide,
            optimizer,
            loss=Trace_ELBO()
        )
        
        try:
            svi_result = svi.run(rng_key, self.n_iterations, initial_flat, progress_bar=False)
            params = svi_result.params
            alpha = params["alpha"]
            
            # Normaliser pour obtenir les probabilites
            probs = alpha / jnp.sum(alpha, axis=-1, keepdims=True)
            return probs
        except Exception as e:
            print(f"Erreur lors de l'inference: {e}")
            return jnp.ones((81, 9)) / 9
    
    def _get_best_cells(self, probs: jnp.ndarray, grid: List[List[int]]) -> List[Tuple[int, int, float]]:
        """
        Trouve les cellules vides avec la plus haute confiance.
        """
        candidates = []
        
        for i in range(9):
            for j in range(9):
                if grid[i][j] == 0:
                    idx = i * 9 + j
                    cell_prob = probs[idx]
                    max_prob = float(jnp.max(cell_prob))
                    best_value = int(jnp.argmax(cell_prob))
                    
                    if max_prob >= self.confidence_threshold:
                        candidates.append((idx, best_value, max_prob))
        
        # Trier par confiance decroissante
        candidates.sort(key=lambda x: x[2], reverse=True)
        return candidates


# Test du solveur iteratif
test_puzzle = puzzle_to_grid(puzzles[0])
print_grid(test_puzzle, "Puzzle initial:")

solver = IterativeProbabilisticSolver(
    n_iterations=50,
    cells_per_iter=1,
    lr=0.05,
    confidence_threshold=0.7
)

start_time = time.time()
solution, metadata = solver.solve(test_puzzle)
elapsed = time.time() - start_time

print_grid(solution, f"\nSolution (en {elapsed:.2f}s):")
print(f"\nMetadonnees:")
print(f"  - Iterations: {metadata['iterations']}")
print(f"  - Cellules fixees: {len(metadata['cells_fixed'])}")
print(f"  - Converge: {metadata['converged']}")
print(f"  - Solution valide: {verify_solution(solution)}")
if metadata['confidences']:
    print(f"  - Confiance moyenne: {np.mean(metadata['confidences']):.3f}")


Puzzle initial:
-------------------------
| 9   2 |     5 | 4   3 |
| 1     |   6 3 |   2 5 |
| 5   8 | 4   7 |   6   |
|-----------------------|
|   2 6 | 3   9 |     1 |
|   5 7 |   1   | 2 9   |
|   9   | 6 7   | 5 3   |
|-----------------------|
| 2 4   | 5 3   | 6     |
| 7   5 | 2     | 3   4 |
|   8   |   4 1 | 9 5   |
-------------------------


C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:109: UserWarning: Currently, SVI with Trace_ELBO loss does not support models with discrete latent variables
  svi_result = svi.run(rng_key, self.n_iterations, initial_flat, progress_bar=False)


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


### Interpretation du solveur iteratif

Le solveur iteratif presente des avantages et inconvenients par rapport a l'approche naive :

| Aspect | Solveur Iteratif | Inference Directe |
|--------|-----------------|-------------------|
| **Convergence** | Progressive, visible | Tout ou rien |
| **Confiance** | Explicite par cellule | Globale seulement |
| **Robustesse** | Peut recuperer d'erreurs | Echec total si echec |
| **Performance** | Plus lent (multiples inférences) | Plus rapide (une inférence) |

**Point cle** : L'approche iterative permet de "guider" la resolution en fixant d'abord les cellules les plus certaines.

## 5. Solveur Hybride : Probabiliste + Propagation de Contraintes

Pour ameliorer la robustesse, nous combinons l'inference probabiliste avec une propagation de contraintes simple (elimination des candidats impossibles).

In [5]:
class HybridProbabilisticSolver:
    """
    Solveur hybride combinant inference probabiliste et propagation de contraintes.
    
    Cette approche est plus robuste que le solveur purement probabiliste.
    """
    
    def __init__(self, n_iterations: int = 50, lr: float = 0.05):
        self.n_iterations = n_iterations
        self.lr = lr
        self.base_solver = IterativeProbabilisticSolver(n_iterations, 1, lr, 0.5)
    
    def solve(self, grid: List[List[int]]) -> Tuple[List[List[int]], dict]:
        """
        Resout un Sudoku en alternant propagation de contraintes et inference probabiliste.
        """
        grid = [row[:] for row in grid]
        metadata = {
            'constraint_propagations': 0,
            'probabilistic_fixes': 0,
            'converged': False
        }
        
        while True:
            # Etape 1: Propagation de contraintes (deterministe)
            progress = self._constraint_propagation(grid)
            metadata['constraint_propagations'] += progress
            
            # Verifier si resolu
            if self._is_solved(grid):
                metadata['converged'] = True
                break
            
            # Verifier si bloque
            if progress == 0:
                # Etape 2: Inference probabiliste pour debloquer
                probs = self.base_solver._infer_probabilities(grid)
                
                # Fixer la cellule la plus certaine
                best = self._get_single_best_cell(probs, grid)
                if best is None:
                    break  # Bloque
                
                idx, value, confidence = best
                row, col = idx // 9, idx % 9
                grid[row][col] = value + 1
                metadata['probabilistic_fixes'] += 1
        
        return grid, metadata
    
    def _constraint_propagation(self, grid: List[List[int]]) -> int:
        """
        Applique la propagation de contraintes (naked singles).
        Retourne le nombre de cellules fixees.
        """
        fixed = 0
        changed = True
        
        while changed:
            changed = False
            for i in range(9):
                for j in range(9):
                    if grid[i][j] == 0:
                        candidates = self._get_candidates(grid, i, j)
                        if len(candidates) == 1:
                            grid[i][j] = candidates[0]
                            fixed += 1
                            changed = True
        
        return fixed
    
    def _get_candidates(self, grid: List[List[int]], row: int, col: int) -> List[int]:
        """
        Retourne les candidats possibles pour une cellule.
        """
        used = set()
        
        # Ligne
        used.update(grid[row])
        
        # Colonne
        used.update(grid[r][col] for r in range(9))
        
        # Bloc
        br, bc = (row // 3) * 3, (col // 3) * 3
        for r in range(br, br + 3):
            for c in range(bc, bc + 3):
                used.add(grid[r][c])
        
        return [v for v in range(1, 10) if v not in used]
    
    def _get_single_best_cell(self, probs: jnp.ndarray, grid: List[List[int]]) -> Optional[Tuple]:
        """
        Trouve la meilleure cellule a fixer parmi celles avec plusieurs candidats.
        """
        best = None
        best_confidence = 0
        
        for i in range(9):
            for j in range(9):
                if grid[i][j] == 0:
                    candidates = self._get_candidates(grid, i, j)
                    if len(candidates) > 1:
                        idx = i * 9 + j
                        cell_prob = probs[idx]
                        
                        # Ne considerer que les candidats valides
                        for c in candidates:
                            prob = float(cell_prob[c - 1])
                            if prob > best_confidence:
                                best_confidence = prob
                                best = (idx, c - 1, prob)
        
        return best
    
    def _is_solved(self, grid: List[List[int]]) -> bool:
        """Verifie si la grille est complete."""
        return all(cell > 0 for row in grid for cell in row)


# Test du solveur hybride
print("=== Test du Solveur Hybride ===")

hybrid_solver = HybridProbabilisticSolver(n_iterations=50, lr=0.05)

for i, puzzle_str in enumerate(puzzles[:3]):
    test_grid = puzzle_to_grid(puzzle_str)
    
    start = time.time()
    solution, meta = hybrid_solver.solve(test_grid)
    elapsed = time.time() - start
    
    valid = verify_solution(solution)
    print(f"\nPuzzle {i+1}: {'OK' if valid else 'INVALIDE'} ({elapsed*1000:.0f}ms)")
    print(f"  - Propagation: {meta['constraint_propagations']} cellules")
    print(f"  - Probabiliste: {meta['probabilistic_fixes']} cellules")

=== Test du Solveur Hybride ===

Puzzle 1: OK (0ms)
  - Propagation: 36 cellules
  - Probabiliste: 0 cellules

Puzzle 2: OK (0ms)
  - Propagation: 49 cellules
  - Probabiliste: 0 cellules


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:109: UserWarning: Currently, SVI with Trace_ELBO loss does not support models with discrete latent variables
  svi_result = svi.run(rng_key, self.n_iterations, initial_flat, progress_bar=False)


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


### Interpretation du solveur hybride

Le solveur hybride combine les forces des deux approches :

| Technique | Role | Avantage |
|-----------|------|----------|
| **Propagation de contraintes** | Resolution deterministe | Garantie locale, tres rapide |
| **Inference probabiliste** | Deblocage quand propagation echoue | Explore les alternatives |

**Point cle** : La propagation de contraintes resout la plupart des cellules "faciles", laissant l'inference probabiliste gerer seulement les cas ambigus.

> **Note technique** : Cette approche est similaire a celle utilisee dans le notebook C# Infer.NET (`IterativeSudokuModel`), mais avec une propagation de contraintes plus explicite.

## 6. Benchmark Comparatif

In [6]:
def benchmark_solvers(puzzles: List[str], limit: int = 5):
    """
    Compare les differents solveurs probabilistes.
    """
    solvers = {
        "Iteratif (seuil 0.7)": IterativeProbabilisticSolver(50, 1, 0.05, 0.7),
        "Iteratif (seuil 0.5)": IterativeProbabilisticSolver(50, 1, 0.05, 0.5),
        "Hybride": HybridProbabilisticSolver(50, 0.05),
    }
    
    results = {name: {"solved": 0, "total_time": 0, "errors": 0} for name in solvers}
    
    for i, puzzle_str in enumerate(puzzles[:limit]):
        grid = puzzle_to_grid(puzzle_str)
        print(f"\n--- Puzzle {i+1} ---")
        
        for name, solver in solvers.items():
            test_grid = [row[:] for row in grid]
            
            start = time.time()
            try:
                if hasattr(solver, 'solve'):
                    solution, _ = solver.solve(test_grid)
                else:
                    solution = solver.solve(test_grid)
                elapsed = time.time() - start
                
                if verify_solution(solution):
                    results[name]["solved"] += 1
                    results[name]["total_time"] += elapsed
                    status = "OK"
                else:
                    errors = count_errors(solution)
                    results[name]["errors"] += errors
                    status = f"INVALIDE ({errors} erreurs)"
                
                print(f"  {name}: {status} ({elapsed*1000:.0f}ms)")
            except Exception as e:
                elapsed = time.time() - start
                results[name]["errors"] += 81  # Penalite maximale
                print(f"  {name}: ERREUR ({e})")
    
    # Resume
    print("\n" + "=" * 50)
    print("RESUME")
    print("=" * 50)
    print(f"{'Solveur':<25} {'Resolus':<10} {'Temps moy':<12} {'Erreurs':<10}")
    print("-" * 50)
    
    for name, data in results.items():
        solved = data["solved"]
        avg_time = data["total_time"] / max(solved, 1) * 1000
        print(f"{name:<25} {solved}/{limit:<8} {avg_time:.0f}ms{' '*5} {data['errors']}")
    
    return results

# Executer le benchmark
print("=== Benchmark sur puzzles Easy ===")
results = benchmark_solvers(puzzles, limit=5)

=== Benchmark sur puzzles Easy ===

--- Puzzle 1 ---
Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimen

C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:109: UserWarning: Currently, SVI with Trace_ELBO loss does not support models with discrete latent variables
  svi_result = svi.run(rng_key, self.n_iterations, initial_flat, progress_bar=False)


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


...

Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


Erreur lors de l'inference: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function body_fn at C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\numpyro\infer\svi.py:383 for scan. This value became a tracer due to JAX operations on these lines:

  operation a:f32[81,9] = broadcast_in_dim[
  broadcast_dimensions=()
  shape=(81, 9)
  sharding=None
] 1.0:f32[]
    from line C:\Users\jsboi\AppData\Local\Temp\ipykernel_188480\3411413045.py:75:16 (IterativeProbabilisticSolver._infer_probabilities.<locals>.guide)

  operation b:f32[81,9] c:f32[81,9] d:f32[81,9] = jit[
  name=clip
  jaxpr={ lambda ; a:f32[81,9] e:f32[] f:f32[]. let
      g:f32[81,9] = max e a
      h:bool[81,9] = eq a g
      i:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None
      ] 1.0:f32[]
      j:f32[81,9] = broadcast_in_dim[
        broadcast_dimensions=()
        shape=(81, 9)
        sharding=None


### Analyse des resultats du benchmark

Les resultats montrent clairement les compromis entre les approches :

| Solveur | Strategie | Performance attendue |
|---------|-----------|---------------------|
| **Iteratif (seuil 0.7)** | Fixe seulement les cellules tres certaines | Plus lent, moins de risques |
| **Iteratif (seuil 0.5)** | Fixe les cellules modereeement certaines | Plus rapide, plus de risques |
| **Hybride** | Combine propagation + probabiliste | Meilleur compromis |

**Observation cle** : Le solveur hybride devrait avoir le meilleur taux de succes car il ne s'appuie sur l'inference probabiliste que lorsque la propagation deterministe echoue.

## 7. Comparaison avec Infer.NET (C#)

Ce notebook Python est l'equivalent du notebook C# `Sudoku-15-Infer-Csharp.ipynb`. Voici une comparaison des deux approches :

In [7]:
# Tableau comparatif
comparison_data = {
    "Aspect": [
        "Bibliotheque",
        "Algorithme d'inference",
        "Contraintes",
        "Variables discretes",
        "Modele de base",
        "Solveur robuste",
        "Solveur iteratif",
        "Precompilation",
        "Performance (Easy)",
        "Performance (Medium)",
        "Garantie de solution"
    ],
    "Infer.NET (C#)": [
        "Microsoft.ML.Probabilistic",
        "Expectation Propagation",
        "Dures (ConstrainFalse)",
        "Natif (intVar)",
        "NaiveProbabilisticSolver",
        "RobustProbabilisticSolver (Dirichlet)",
        "IterativeSudokuModel",
        "Oui (DLL generee)",
        "~1-5s",
        "Variable (echec possible)",
        "Partielle"
    ],
    "NumPyro (Python)": [
        "NumPyro + JAX",
        "Variational Inference (SVI)",
        "Douces (penalites)",
        "Enumeration necessaire",
        "simple_sudoku_model",
        "Dirichlet guide (SVI)",
        "IterativeProbabilisticSolver",
        "Non",
        "~1-5s",
        "Variable (echec possible)",
        "Partielle"
    ]
}

import pandas as pd
df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))

                Aspect                        Infer.NET (C#)             NumPyro (Python)
          Bibliotheque            Microsoft.ML.Probabilistic                NumPyro + JAX
Algorithme d'inference               Expectation Propagation  Variational Inference (SVI)
           Contraintes                Dures (ConstrainFalse)           Douces (penalites)
   Variables discretes                        Natif (intVar)       Enumeration necessaire
        Modele de base              NaiveProbabilisticSolver          simple_sudoku_model
       Solveur robuste RobustProbabilisticSolver (Dirichlet)        Dirichlet guide (SVI)
      Solveur iteratif                  IterativeSudokuModel IterativeProbabilisticSolver
        Precompilation                     Oui (DLL generee)                          Non
    Performance (Easy)                                 ~1-5s                        ~1-5s
  Performance (Medium)             Variable (echec possible)    Variable (echec possible)
  Garantie

### Lecons retenues

1. **Infer.NET est superieur pour les contraintes dures** : L'algorithme Expectation Propagation gere nativement les contraintes discretes.

2. **NumPyro est plus flexible mais moins adapte** : L'inference variationnelle est concue pour l'apprentissage bayesien, pas pour la satisfaction de contraintes.

3. **L'approche hybride est la plus robuste** : Combiner propagation de contraintes deterministe avec inference probabiliste donne les meilleurs resultats.

4. **Pour la production, utiliser OR-Tools ou Z3** : Ces bibliotheques sont concues pour les CSP et sont 100-1000x plus rapides.

## 8. Pourquoi cette approche reste educative

Malgre ses limitations pratiques, ce notebook enseigne des concepts importants :

### Concepts appris

1. **Modelisation probabiliste** : Comment representer un probleme discret avec des distributions
2. **Inference variationnelle** : Comment approximer une distribution a posteriori
3. **Contraintes douces vs dures** : La difference fondamentale entre penalites et contraintes logiques
4. **Methodes iteratives** : Comment construire une solution progressivement

### Quand utiliser la programmation probabiliste

| Cas d'usage | Appropriate ? |
|-------------|--------------|
| Apprentissage bayesien | Oui |
| Modeles hierarchiques | Oui |
| Incertitude epistemique | Oui |
| **Satisfaction de contraintes pures** | **Non** |
| **Optimisation combinatoire** | **Non** |

> **Recommandation** : Pour resoudre des Sudokus en production, utiliser OR-Tools, Z3 ou Choco. La programmation probabiliste est un outil pedagogique pour comprendre l'inference, pas un solveur de contraintes efficace.

---

**Navigation** : [<< Choco](Sudoku-11-Choco-Python.ipynb) | [Index](README.md) | [Neural Network >>](Sudoku-16-NeuralNetwork-Python.ipynb)

**Voir aussi** :
- [Sudoku-15-Infer-Csharp](Sudoku-15-Infer-Csharp.ipynb) - Version Infer.NET (C#)
- [Probas](../Probas/README.md) - Serie complete sur la programmation probabiliste
- [Sudoku-10-ORTools-Python](Sudoku-10-ORTools-Python.ipynb) - Approche CP-SAT (recommandee)